In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [97]:
!pip3 install pyvi keras gensim
!pip3 install --upgrade keras tensorflow
!pip3 install "tensorflow-text==2.13.*"

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-3.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-macosx_11_0_arm64.whl (110 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.13.* (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.13.*


In [98]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
# from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import re
from string import digits
from collections import Counter
from pyvi import ViTokenizer

In [99]:
data_train = pd.read_csv("./vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("./vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [100]:
data_train.head()

,Class,Data
0,-1,"Mình đã dùng anywhere thế hệ đầu, quả là đầy t..."
1,-1,"Quan tâm nhất là độ trễ có cao không, dùng thi..."
2,-1,"dag xài con cùi bắp 98k....pin trâu, mỗi tội đ..."
3,-1,logitech chắc hàng phải tiền triệu trở lên dùn...
4,-1,"Đang xài con m175 cùi mía , nhà xài nhiều chuộ..."


In [101]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [102]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

In [103]:
with open("./stopwords.txt", "r", encoding="utf8") as f:
    stopwords = f.read().splitlines()

def remove_stop_words(text, stopwords):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    text = ' '.join(words)
    return text

In [104]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [105]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in string.digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed.append(review_cool_one)

In [106]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())
   

In [107]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [108]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [109]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [110]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [111]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [112]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format("./vi-model-CBOW.bin", binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [113]:
from keras.models import Model
from keras.layers import *
from keras import regularizers
from tensorflow.keras.optimizers import Adam

sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = Embedding(input_dim=vocabulary_size, output_dim=EMBEDDING_DIM, input_length=sequence_length)(inputs)

conv_0 = Conv1D(num_filters, filter_sizes[0], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_0 = GlobalMaxPooling1D()(conv_0)

conv_1 = Conv1D(num_filters, filter_sizes[1], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_1 = GlobalMaxPooling1D()(conv_1)

conv_2 = Conv1D(num_filters, filter_sizes[2], activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
maxpool_2 = GlobalMaxPooling1D()(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2])
dropout = Dropout(drop)(merged_tensor)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

model = Model(inputs, output)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 300)]                0         []                            
                                                                                                  
 embedding_5 (Embedding)     (None, 300, 400)             3167600   ['input_11[0][0]']            
                                                                                                  
 conv1d_19 (Conv1D)          (None, 298, 100)             120100    ['embedding_5[0][0]']         
                                                                                                  
 conv1d_20 (Conv1D)          (None, 297, 100)             160100    ['embedding_5[0][0]']         
                                                                                            

In [114]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

model.fit(data, labels, validation_split=0.2,
          epochs=10, batch_size=256, callbacks=callbacks_list, shuffle=True)

Epoch 1/10
16/16 [==============================] - 13s 784ms/step - loss: 4.3601 - accuracy: 0.4522 - val_loss: 3.7530 - val_accuracy: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 13s 808ms/step - loss: 2.2006 - accuracy: 0.6064 - val_loss: 2.3455 - val_accuracy: 0.0000e+00
Epoch 3/10
16/16 [==============================] - 13s 821ms/step - loss: 1.3519 - accuracy: 0.6544 - val_loss: 1.9357 - val_accuracy: 0.0000e+00
Epoch 4/10
16/16 [==============================] - 13s 835ms/step - loss: 1.0434 - accuracy: 0.6782 - val_loss: 1.8663 - val_accuracy: 0.0000e+00
Epoch 5/10
16/16 [==============================] - 13s 834ms/step - loss: 0.9011 - accuracy: 0.7042 - val_loss: 1.7912 - val_accuracy: 0.0000e+00
Epoch 6/10
16/16 [==============================] - 13s 842ms/step - loss: 0.8133 - accuracy: 0.7358 - val_loss: 1.7834 - val_accuracy: 0.0059
Epoch 7/10
16/16 [==============================] - 14s 867ms/step - loss: 0.7400 - accuracy: 0.7748 - val_loss: 1.7700 - 

In [115]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [116]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [117]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    # review_cool_one = remove_stop_words(review_cool_one, stopwords)
    reviews_processed_test.append(review_cool_one)

In [118]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [119]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [120]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [121]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 1s 29ms/step - loss: 1.0589 - accuracy: 0.6181


In [122]:
print("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


loss: 105.89%
accuracy: 61.81%
